
[![Open in Colb](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/frkr/gemini-sace/blob/master/PDF_Alura_Google.ipynb)

In [ ]:
# @title Crie sua Api Key (chaves da aplicação) neste link: [Api Key](https://aistudio.google.com/app/apikey?hl=pt-br) { run: "auto", vertical-output: true, display-mode: "form" }
GOOGLE_API_KEY = "" # @param {type:"string"}
MODEL_NAME = "gemini-1.0-pro" # @param {type:"string"}
#"gemini-1.5-pro-latest"


!pip install -q google-generativeai

import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import hashlib

genai.configure(api_key=GOOGLE_API_KEY)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#to_markdown("# simple text")

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name=MODEL_NAME,
                              generation_config=generation_config,
                              safety_settings=safety_settings)

MODEL_NAME

# Inundações da Bacia do Taquari.

**O sistema vai baixar boletins de 2022 e 2023 e vamos tentar descobrir se o Gemini consegue fazer previsões.**

---
- Sistemas de Alerta Hidrológico (SAHs) - [SACE](https://www.sgb.gov.br/sace/)
- Bacia -  [Taquari, Porto Alegre e Lagoa dos Patos](https://www.sgb.gov.br/sace/index_bacias_monitoradas.php?getbacia=btaquari#)
- [Boletins](https://www.sgb.gov.br/sace/boletins.php?idbacia=9)


In [ ]:
# @title Download { run: "auto", vertical-output: true, display-mode: "form" }
LINK_BASE = "https://www.sgb.gov.br/sace/" # @param {type:"string"}
ARQUIVOS = "https://www.sgb.gov.br/sace/boletins.php?idbacia=9" # @param {type:"string"}
FILTRO1 = "2023" # @param {type:"string"}
FILTRO2 = "2022" # @param {type:"string"}


from urllib import request
from bs4 import BeautifulSoup
import re
import os
import urllib
from urllib.parse import urlparse

# connect to website and get list of all pdfs
response = request.urlopen(ARQUIVOS).read()
soup= BeautifulSoup(response, "html.parser")
links = soup.find_all('a', href=re.compile(r'(.pdf)'))

# clean the pdf link names
url_list = []
for el in links:
  if(el['href'].startswith('http')):
    url_list.append(el['href'])
  else:
    url_list.append(LINK_BASE + el['href'])

# download the pdfs to a specified location
downloaded = []
for url in url_list:
    # print(url)
    if FILTRO1 in url or FILTRO2 in url:
      downloaded.append(os.path.basename(url))
      print(os.path.basename(url))
      request.urlretrieve(url.replace(" ","%20"), os.path.basename(url))

print('download concluido')

20240511_08-20240511 - 104901.pdf
20240511_02-20240511 - 032459.pdf
20240506_22-20240506 - 234505.pdf
20240506_15-20240506 - 164641.pdf
20240506_07-20240506 - 083615.pdf
20240505_22-20240506 - 002534.pdf
20240505_15-20240505 - 163823.pdf
20240505_06-20240505 - 082602.pdf
20240504_22-20240505 - 005132.pdf
20240504_08-20240504 - 101433.pdf
20240503_22-20240503 - 232910.pdf
20240503_15-20240503 - 172717.pdf
20240503_06-20240503 - 082220.pdf
20240503_02-20240503 - 031646.pdf
20240502_22-20240502 - 232754.pdf
20240502_18-20240502 - 193408.pdf
20240502_14-20240502 - 155032.pdf
20240502_10-20240502 - 144947.pdf
20240502_06-20240502 - 091143.pdf
20240502_02-20240502 - 034816.pdf
20240501_22-20240502 - 002838.pdf
20240501_14-20240501 - 162844.pdf
20240501_10-20240501 - 111848.pdf
20240501_06-20240501 - 073914.pdf
20240501_02-20240501 - 035231.pdf
20240430_22-20240430 - 232725.pdf
20240430_19-20240430 - 215222.pdf
20240430_14-20240430 - 164218.pdf
20240430_10-20240430 - 114943.pdf
20240430_06-20

In [ ]:
#@title Criar historico { run: "auto", vertical-output: true, display-mode: "form" }

history = []
for url in downloaded:
  with open(os.path.basename(url),"rb") as f:
    image_blob = f.read()
    print(os.path.basename(url))
    history.append({
        "role": "user",
        "parts": [{
            "data": image_blob,
            "mime_type": "application/pdf",
        }]
    })

convo = model.start_chat(history=history)

print('Arquivos enviados')


In [ ]:
#@title Debater sobre o documento: { run: "auto", vertical-output: true, display-mode: "form" }
prompt = "O que significa estes documentos?" # @param {type:"string"}

convo.send_message(prompt)
to_markdown(convo.last.text)
